<H1> InfiniBox CSI driver demo </H1>
Contact: Gregory Touretsky, @gregnsk

<H2> Install the driver using OpenShift Operator </H2>
<img src=https://github.com/gregnsk/infinibox_csi_driver_demo/blob/master/openshift-operator.png?raw=true width=50%>

<B>Notes</B>
<li>Infinidat CSI driver is supported starting from Kubernetes 1.14 (OpenShift 4.2)
<li>For earlier versions, customers may use Infinidat Dynamic Provisioner for Kubernetes (supported since Kubernetes 1.6)
<li>Kubernetes versions prior to 1.17 (OpenShift 4.4): some of CSI features like snapshots, clones, or raw block volumes might not be supported or enabled by default. Refer to relevant documentation for more details on feature-gate enablement
<li>Examples in this document use kubectl command, the same commands will work with OpenShift "oc" interface
<li>Infinidat CSI driver can be installed using Helm Chart or OpenShift Operator

In [88]:
kubectl get nodes

NAME          STATUS   ROLES    AGE   VERSION
gtouret-k51   Ready    master   44d   v1.18.0
gtouret-k52   Ready    <none>   44d   v1.18.0
io307         Ready    <none>   44d   v1.18.0


In [89]:
# We recommend to install CSI driver in a dedicated namespace. Here it's running in a namespace "ibox"
# There is a single Controller (csi-infinibox-driver) instance per cluster 
# and a single Node (csi-infinibox-node) instance per worker node 
kubectl get pod -n ibox -o wide

NAME                       READY   STATUS    RESTARTS   AGE   IP              NODE          NOMINATED NODE   READINESS GATES
csi-infinibox-driver-0     5/5     Running   0          25d   10.244.2.55     io307         <none>           <none>
csi-infinibox-node-85g4t   2/2     Running   0          25d   172.20.87.214   gtouret-k51   <none>           <none>
csi-infinibox-node-jw9hx   2/2     Running   0          25d   172.20.87.99    gtouret-k52   <none>           <none>
csi-infinibox-node-qspr4   2/2     Running   0          25d   172.20.78.63    io307         <none>           <none>


In [90]:
# In this demo PVCs and pods will be created in a namespace "demo"
# At the beginning of the demo this namespace is empty
kubectl get pvc,volumesnapshot,pod -n demo



No resources found.


<H2>Create a storage class <H2>

In [91]:
cat nfs/storageclass.yaml

apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: ibox-nfs-storageclass-demo
provisioner: infinibox-csi-driver
reclaimPolicy: Delete
volumeBindingMode: Immediate
allowVolumeExpansion: true
parameters:
    pool_name: k8s_csi
    network_space: nas1
    provision_type: THIN    
    storage_protocol: nfs   
    nfs_mount_options: hard,rsize=1048576,wsize=1048576
    nfs_export_permissions : "[{'access':'RW','client':'172.20.1.1-172.20.255.255','no_root_squash':true}]"
    ssd_enabled: "true"
    csi.storage.k8s.io/provisioner-secret-name: infinibox-creds
    csi.storage.k8s.io/provisioner-secret-namespace: ibox
    csi.storage.k8s.io/controller-publish-secret-name: infinibox-creds
    csi.storage.k8s.io/controller-publish-secret-namespace: ibox
    csi.storage.k8s.io/node-stage-secret-name: infinibox-creds
    csi.storage.k8s.io/node-stage-secret-namespace: ibox
    csi.storage.k8s.io/node-publish-secret-name: infinibox-creds
    csi.storage.k8s.io/node-publish-secret-nam

In [92]:
# Storage class defines InfiniBox pool for all PVs, NFS netspace nas1 and some other parameters
# Use infinishell to query pool details
./infinishell pool.query name=k8s_csi --columns=name,virtual_total,physical_total
# Use infinishell to query netspace details
./infinishell config.net_space.query net_space=nas1
./infinishell config.net_space.ip.query net_space=nas1

Executing pool.query name=k8s_csi --columns=name,virtual_total,physical_total on ibox2811...
NAME                         VIRTUAL TOTAL        PHYSICAL TOTAL
k8s_csi                          200.00 TB               1.00 TB
Executing config.net_space.query net_space=nas1 on ibox2811...
NAME  SERVICE  NETWORK         MTU   RATE LIMIT  GATEWAY        IPS  INTERFACES
nas1  NAS      172.20.32.0/19  9000           -  172.20.63.254  3    pg_data1  
Executing config.net_space.ip.query net_space=nas1 on ibox2811...
NETWORK SPACE  IP ADDRESS    ENABLED  NODE  NETWORK INTERFACE  TYPE
nas1           172.20.37.53  yes      1     pg_data1           NAS 
nas1           172.20.37.56  yes      2     pg_data1           NAS 
nas1           172.20.37.58  yes      3     pg_data1           NAS 


In [93]:
kubectl create -f nfs/storageclass.yaml

storageclass.storage.k8s.io/ibox-nfs-storageclass-demo created


In [94]:
kubectl get sc ibox-nfs-storageclass-demo

NAME                         PROVISIONER            RECLAIMPOLICY   VOLUMEBINDINGMODE   ALLOWVOLUMEEXPANSION   AGE
ibox-nfs-storageclass-demo   infinibox-csi-driver   Delete          Immediate           true                   2s


<H2> Create a Persistent Volume Claim (PVC) </H2>

In [95]:
cat nfs/pvc.yaml

apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: ibox-pvc-demo
  namespace: demo
spec:
  accessModes:
  - ReadWriteMany
  resources:
    requests:
      storage: 1Gi
  storageClassName: ibox-nfs-storageclass-demo


In [96]:
kubectl create -f nfs/pvc.yaml

persistentvolumeclaim/ibox-pvc-demo created


In [97]:
kubectl get pvc ibox-pvc-demo -n demo

NAME            STATUS   VOLUME           CAPACITY   ACCESS MODES   STORAGECLASS                 AGE
ibox-pvc-demo   Bound    csi-0a88785e41   1Gi        RWX            ibox-nfs-storageclass-demo   4s


In [98]:
PV=`kubectl get pvc ibox-pvc-demo -n demo | grep ibox-pvc-demo | awk '{print $3}'`
kubectl get pv $PV

NAME             CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS   CLAIM                STORAGECLASS                 REASON   AGE
csi-0a88785e41   1Gi        RWX            Delete           Bound    demo/ibox-pvc-demo   ibox-nfs-storageclass-demo            9s


In [99]:
# Use infinishell to query InfiniBox filesystem details
./infinishell fs.query name=$PV --columns=name,size,used,pool,data_reduction

Executing fs.query name=csi-0a88785e41 --columns=name,size,used,pool,data_reduction on ibox2811...
NAME                                  SIZE                  USED  POOL        DATA REDUCTION
csi-0a88785e41                     1.07 GB               1.63 MB  k8s_csi          16.00 : 1


<H2> Create a snapshot class </H2>

In [100]:
cat nfs/snapshotclass.yaml

apiVersion: snapshot.storage.k8s.io/v1alpha1
kind: VolumeSnapshotClass
metadata:
  name: ibox-snapshotclass-demo
snapshotter: infinibox-csi-driver
parameters:
  csi.storage.k8s.io/snapshotter-secret-name: infinibox-creds
  csi.storage.k8s.io/snapshotter-secret-namespace: ibox



In [101]:
kubectl create -f nfs/snapshotclass.yaml

volumesnapshotclass.snapshot.storage.k8s.io/ibox-snapshotclass-demo created


In [102]:
kubectl get volumesnapshotclass

NAME                      AGE
ibox-snapshotclass-demo   2s


<H2> Create a snapshot </H2>

In [103]:
cat nfs/snapshot.yaml

apiVersion: snapshot.storage.k8s.io/v1alpha1
kind: VolumeSnapshot
metadata:
  name: ibox-pvc-snapshot-demo
  namespace: demo
spec:
  snapshotClassName: ibox-snapshotclass-demo
  source:
    name: ibox-pvc-demo
    kind: PersistentVolumeClaim



In [104]:
kubectl create -f nfs/snapshot.yaml

volumesnapshot.snapshot.storage.k8s.io/ibox-pvc-snapshot-demo created


In [105]:
kubectl get volumesnapshot ibox-pvc-snapshot-demo -n demo

NAME                     AGE
ibox-pvc-snapshot-demo   4s


In [106]:
kubectl describe volumesnapshot ibox-pvc-snapshot-demo  -n demo | grep snapcontent

  Snapshot Content Name:  snapcontent-24ed6d14-8c7b-4137-a2c5-de8765775978


In [107]:
VSC=`kubectl describe volumesnapshot ibox-pvc-snapshot-demo  -n demo | grep snapcontent | awk '{print $4}'`
kubectl get volumesnapshotcontent $VSC

NAME                                               AGE
snapcontent-24ed6d14-8c7b-4137-a2c5-de8765775978   16s


In [108]:
# Use infinishell to query InfiniBox snapshot details. Snapshot name is part of the snapcontent ID
SNAP=`echo $VSC | cut -b1-23 | sed 's/-//g' | sed 's/snapcontent/csi-/'`

./infinishell fs.snap.query name=$SNAP --columns=name,size,allocated,wp,pool

Executing fs.snap.query name=csi-24ed6d148c --columns=name,size,allocated,wp,pool on ibox2811...
NAME                                  SIZE             ALLOCATED  WP   POOL      
csi-24ed6d148c                     1.07 GB               4.09 KB  yes  k8s_csi   


<H2> Create a new PVC from the snapshot (restore) </H2>

In [109]:
cat nfs/restoresnapshot.yaml

apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: ibox-snapshot-pvc-restore-demo
  namespace: demo
spec:
  storageClassName: ibox-nfs-storageclass-demo
  dataSource:
    name: ibox-pvc-snapshot-demo
    kind: VolumeSnapshot
    apiGroup: "snapshot.storage.k8s.io"
  accessModes:
    - ReadWriteMany
  resources:
    requests:
      storage: 1Gi



In [110]:
kubectl create -f nfs/restoresnapshot.yaml

persistentvolumeclaim/ibox-snapshot-pvc-restore-demo created


In [111]:
kubectl get pvc ibox-snapshot-pvc-restore-demo -n demo

NAME                             STATUS   VOLUME           CAPACITY   ACCESS MODES   STORAGECLASS                 AGE
ibox-snapshot-pvc-restore-demo   Bound    csi-0472a03a96   1Gi        RWX            ibox-nfs-storageclass-demo   4s


In [112]:
# Use infinishell to query restore dataset details on Infinibox
RESTORE=`kubectl get pvc ibox-snapshot-pvc-restore-demo -n demo | grep ibox-snapshot-pvc-restore-demo| awk '{print $3}'`
./infinishell fs.snap.query name=$RESTORE --columns=name,size,allocated,wp,pool

Executing fs.snap.query name=csi-0472a03a96 --columns=name,size,allocated,wp,pool on ibox2811...
NAME                                  SIZE             ALLOCATED  WP   POOL      
csi-0472a03a96                     1.07 GB               4.09 KB  no   k8s_csi   


<H2> Create a new PVC from the existing PVC (clone) </H2>

In [113]:
cat nfs/clonepvc.yaml

apiVersion: v1
kind: PersistentVolumeClaim
metadata:
    name: ibox-pvc-clone-demo
    namespace: demo
spec:
  accessModes:
  - ReadWriteOnce
  storageClassName: ibox-nfs-storageclass-demo
  resources:
    requests:
      storage: 1Gi
  dataSource:
    kind: PersistentVolumeClaim
    name: ibox-pvc-demo


In [114]:
kubectl create -f nfs/clonepvc.yaml

persistentvolumeclaim/ibox-pvc-clone-demo created


In [115]:
kubectl get pvc ibox-pvc-clone-demo -n demo

NAME                  STATUS   VOLUME           CAPACITY   ACCESS MODES   STORAGECLASS                 AGE
ibox-pvc-clone-demo   Bound    csi-7c81e147af   1Gi        RWO            ibox-nfs-storageclass-demo   2s


In [116]:
# Use infinishell to query clone dataset details on Infinibox
CLONE=`kubectl get pvc ibox-pvc-clone-demo -n demo | grep ibox-pvc-clone-demo| awk '{print $3}'`
./infinishell fs.snap.query name=$CLONE --columns=name,size,allocated,wp,pool

Executing fs.snap.query name=csi-7c81e147af --columns=name,size,allocated,wp,pool on ibox2811...
NAME                                  SIZE             ALLOCATED  WP   POOL      
csi-7c81e147af                     1.07 GB               4.09 KB  no   k8s_csi   


<H2> Create a pod with a PVC attached </H2>

In [117]:
cat nfs/app.yaml

kind: Pod
apiVersion: v1
metadata:
  name: ibox-pod-pvc-demo
  namespace: demo
spec:
  containers:
    - name: my-frontend
      image: busybox
      volumeMounts:
      - mountPath: "/tmp/data"
        name: ibox-csi-volume
      command: [ "sleep", "1000" ]
  volumes:
    - name: ibox-csi-volume
      persistentVolumeClaim:
        claimName: ibox-pvc-demo


In [118]:
kubectl create -f nfs/app.yaml

pod/ibox-pod-pvc-demo created


In [120]:
kubectl get pod ibox-pod-pvc-demo -n demo

NAME                READY   STATUS    RESTARTS   AGE
ibox-pod-pvc-demo   1/1     Running   0          20s


In [121]:
kubectl exec -it ibox-pod-pvc-demo -n demo -- df -h /tmp/data

Filesystem                Size      Used Available Use% Mounted on
172.20.37.58:/csi-0a88785e41
                          1.0G      1.0M   1023.0M   0% /tmp/data


<H2>Deploying mySQL server with Helm using InfiniBox CSI driver for iSCSI</H2>

In [122]:
cat iscsi/storageclass.yaml

apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: ibox-iscsi-storageclass-demo
provisioner: infinibox-csi-driver
reclaimPolicy: Delete
volumeBindingMode: Immediate
allowVolumeExpansion: true
parameters:
  csi.storage.k8s.io/provisioner-secret-name: infinibox-creds
  csi.storage.k8s.io/provisioner-secret-namespace: ibox
  csi.storage.k8s.io/controller-publish-secret-name: infinibox-creds
  csi.storage.k8s.io/controller-publish-secret-namespace: ibox
  csi.storage.k8s.io/node-stage-secret-name: infinibox-creds
  csi.storage.k8s.io/node-stage-secret-namespace: ibox
  csi.storage.k8s.io/node-publish-secret-name: infinibox-creds
  csi.storage.k8s.io/node-publish-secret-namespace: ibox
  csi.storage.k8s.io/controller-expand-secret-name: infinibox-creds
  csi.storage.k8s.io/controller-expand-secret-namespace: ibox
  useCHAP: "mutual_chap" # none / chap / mutual_chap
  fstype: ext4
  pool_name: "k8s_csi"
  network_space: "iscsi1"
  provision_type: "THIN"
  storage_protocol: "i

In [123]:
kubectl create -f iscsi/storageclass.yaml

storageclass.storage.k8s.io/ibox-iscsi-storageclass-demo created


In [124]:
helm install mysqldemo --set persistence.storageClass=ibox-iscsi-storageclass-demo,nodeSelector.protocol=iscsi stable/mysql

NAME: mysqldemo
LAST DEPLOYED: Mon May 18 14:23:34 2020
NAMESPACE: default
STATUS: deployed
REVISION: 1
NOTES:
MySQL can be accessed via port 3306 on the following DNS name from within your cluster:
mysqldemo.default.svc.cluster.local

To get your root password run:

    MYSQL_ROOT_PASSWORD=$(kubectl get secret --namespace default mysqldemo -o jsonpath="{.data.mysql-root-password}" | base64 --decode; echo)

To connect to your database:

1. Run an Ubuntu pod that you can use as a client:

    kubectl run -i --tty ubuntu --image=ubuntu:16.04 --restart=Never -- bash -il

2. Install the mysql client:

    $ apt-get update && apt-get install mysql-client -y

3. Connect using the mysql cli, then provide your password:
    $ mysql -h mysqldemo -p

To connect to your database directly from outside the K8s cluster:
    MYSQL_HOST=127.0.0.1
    MYSQL_PORT=3306

    # Execute the following command to route the connection:
    kubectl port-forward svc/mysqldemo 3306

    mysql -h ${MYSQL_HOST} -P$

In [125]:
kubectl get deployment mysqldemo

NAME        READY   UP-TO-DATE   AVAILABLE   AGE
mysqldemo   1/1     1            1           62s


In [126]:
kubectl get pvc mysqldemo

NAME        STATUS   VOLUME           CAPACITY   ACCESS MODES   STORAGECLASS                   AGE
mysqldemo   Bound    csi-327885dafc   8Gi        RWO            ibox-iscsi-storageclass-demo   66s


In [127]:
POD=`kubectl get pod | grep mysqldemo | awk '{print $1}'`
kubectl exec -it $POD -- df /var/lib/mysql

Filesystem          1K-blocks   Used Available Use% Mounted on
/dev/mapper/mpathdq   8191416 251852   7923180   4% /var/lib/mysql


In [128]:
# Use infinishell to query InfiniBox volume details
VOL=`kubectl get pvc mysqldemo | grep mysqldemo | awk '{print $3}'`
./infinishell vol.query vol=$VOL --columns=name,size,used,pool,data_reduction

Executing vol.query vol=csi-327885dafc --columns=name,size,used,pool,data_reduction on ibox2811...
NAME                                  SIZE                  USED  POOL        DATA REDUCTION
csi-327885dafc                     8.59 GB             316.40 MB  k8s_csi           3.67 : 1


<H2> Clean everything </H2>

In [129]:
helm uninstall mysqldemo

release "mysqldemo" uninstalled


In [130]:
kubectl delete -f nfs/app.yaml

pod "ibox-pod-pvc-demo" deleted


In [131]:
kubectl delete -f nfs/clonepvc.yaml

persistentvolumeclaim "ibox-pvc-clone-demo" deleted


In [132]:
kubectl delete -f nfs/restoresnapshot.yaml

persistentvolumeclaim "ibox-snapshot-pvc-restore-demo" deleted


In [133]:
kubectl delete -f nfs/snapshot.yaml

volumesnapshot.snapshot.storage.k8s.io "ibox-pvc-snapshot-demo" deleted


In [134]:
kubectl delete -f nfs/snapshotclass.yaml

volumesnapshotclass.snapshot.storage.k8s.io "ibox-snapshotclass-demo" deleted


In [135]:
kubectl delete -f nfs/pvc.yaml

persistentvolumeclaim "ibox-pvc-demo" deleted


In [136]:
kubectl delete -f nfs/storageclass.yaml

storageclass.storage.k8s.io "ibox-nfs-storageclass-demo" deleted


In [137]:
kubectl delete -f iscsi/storageclass.yaml

storageclass.storage.k8s.io "ibox-iscsi-storageclass-demo" deleted
